# Properties and statistics

In this notebook, we will explore how to access properties and statistics of the nodes and edges. We will
* Demonstrate the capabilities of `NodeView` and `EdgeView`
* Present the statistics interface for accessing node/edge statistics

In [1]:
import xgi

In [4]:
H = xgi.load_xgi_data("kaggle-whats-cooking")
print(H)

Hypergraph named kaggle-whats-cooking with 6714 nodes and 39774 hyperedges


## NodeView and EdgeView

*Views* are ways of offering read-only access to the nodes, edges, and their properties. For example, one can
* See what nodes are contained in an edge
* See which edges are maximal
* Find isolated nodes
* Access nodal attributes

For example:

In [7]:
H.edges.singletons()

EdgeView(('940', '2088', '6787', '7011', '8181', '8852', '8990', '10506', '13178', '17804', '18136', '18324', '21008', '22119', '22387', '23512', '26887', '29294', '30636', '32105', '34531', '37220'))

In [10]:
H.edges.members("2088")

{'2974'}

In [11]:
H.nodes["2974"]

{'name': 'promise buttery spread'}

In [19]:
H.edges - H.edges.maximal()

EdgeView(('18', '21', '67', '84', '88', '143', '203', '215', '221', '248', '268', '284', '298', '328', '336', '337', '347', '369', '399', '402', '423', '487', '509', '560', '561', '568', '581', '601', '631', '671', '759', '791', '841', '870', '903', '905', '924', '940', '953', '970', '996', '1028', '1081', '1094', '1151', '1160', '1175', '1177', '1247', '1321', '1334', '1342', '1343', '1383', '1388', '1399', '1434', '1474', '1500', '1510', '1512', '1524', '1557', '1651', '1685', '1699', '1743', '1758', '1767', '1790', '1791', '1817', '1818', '1820', '1844', '1860', '1866', '1905', '1923', '1939', '1943', '1951', '1960', '1962', '2002', '2018', '2060', '2075', '2088', '2104', '2130', '2145', '2190', '2191', '2196', '2226', '2238', '2251', '2260', '2276', '2298', '2311', '2372', '2407', '2420', '2428', '2464', '2501', '2507', '2553', '2565', '2578', '2579', '2581', '2642', '2656', '2672', '2691', '2767', '2774', '2779', '2806', '2827', '2844', '2962', '2983', '2994', '3053', '3054', '305

In [22]:
H.edges.members("67")

{'1611', '1910', '5408', '5700', '6055', '6126'}

In [24]:
[H.nodes[n]["name"] for n in H.edges.members("67")]

['Quorn crumbles',
 'char fillets',
 'dried oregano',
 'chicken thighs',
 'coke zero']

## Statistics

You may have noticed that most of the functionality in the `Hypergraph` and `SimplicialComplex` classes takes care of modifying the unerlying structure of the network, and that these classes provide very limited functionality to compute statistics (a.k.a. measures) from the network. This is done via the `stats` package, explored here.

The stats package is one of the features that sets `xgi` apart from other libraries.  It
provides a common interface to all statistics that can be computed from a network, its
nodes, or edges.

### Introduction to Stat objects

Consider the degree of the nodes of a network `H`.  After computing the values of the
degrees, one may wish to store them in a dict, a list, an array, a dataframe, etc.
Through the stats package, `xgi` provides a simple interface that seamlessly allows for
this type conversion.  This is done via the `NodeStat` class.

In [1]:
import xgi

H = xgi.Hypergraph([[1, 2, 3], [2, 3, 4, 5], [3, 4, 5]])
H.nodes.degree

NodeStat('degree')

This `NodeStat` object is essentially a wrapper over a function that computes the
degrees of all nodes.  One of the main features of `NodeStat` objects is lazy
evaluation: `H.nodes.degree` will not compute the degrees of nodes until a specific
output format is requested.

In [2]:
H.nodes.degree.asdict()

{1: 1, 2: 2, 3: 3, 4: 2, 5: 2}

In [3]:
H.nodes.degree.aslist()

[1, 2, 3, 2, 2]

In [4]:
H.nodes.degree.asnumpy()

array([1, 2, 3, 2, 2])

To compute the degrees of a subset of the nodes, call `degree` from a smaller `NodeView`.

In [5]:
H.nodes([3, 4, 5]).degree.asdict()

{3: 3, 4: 2, 5: 2}

Alternatively, to compute the degree of a single node, use square brackets.

In [6]:
H.nodes.degree[4]

2

Make sure the accessed node is in the underlying view.

In [7]:
# This will raise an exception
# because node 4 is not in the view [1, 2, 3]
#
# H.nodes([1, 2, 3]).degree[4]
#

args and kwargs may be passed to `NodeStat` objects, which will be stored and used when
the evaluation finally takes place.  For example, use the `order` keyword of `degree` to
count only those edges of the specified order.

In [8]:
H.nodes.degree(order=3)

NodeStat('degree', kwargs={'order': 3})

In [9]:
H.nodes.degree(order=3).aslist()

[0, 1, 1, 1, 1]

The stats package provides some convenience functions for numerical operations.

In [10]:
H.nodes.degree.max(), H.nodes.degree.min()

(3, 1)

In [11]:
import numpy as np

st = H.nodes([1, 2, 3]).degree(order=3)
np.round([st.max(), st.min(), st.mean(), st.median(), st.var(), st.std()], 3)

array([1.   , 0.   , 0.667, 1.   , 0.222, 0.471])

As a convenience, each node statistic may also be accessed directly through the network itself.

In [12]:
H.degree()

{1: 1, 2: 2, 3: 3, 4: 2, 5: 2}

Note however that `H.degree` is a method that simply returns a dict, not a `NodeStat` object and thus does not support the features discussed above.

## Node attributes

Node attributes can be conceived of as a node-object mapping and thus they can also be accessed using the `NodeStat` interface and all its funcitonality.

In [13]:
H.add_nodes_from(
    [
        (1, {"color": "red", "name": "horse"}),
        (2, {"color": "blue", "name": "pony"}),
        (3, {"color": "yellow", "name": "zebra"}),
        (4, {"color": "red", "name": "orangutan", "age": 20}),
        (5, {"color": "blue", "name": "fish", "age": 2}),
    ]
)

Access all attributes of all nodes by specifying a return type.

In [14]:
H.nodes.attrs.asdict()

{1: {'color': 'red', 'name': 'horse'},
 2: {'color': 'blue', 'name': 'pony'},
 3: {'color': 'yellow', 'name': 'zebra'},
 4: {'color': 'red', 'name': 'orangutan', 'age': 20},
 5: {'color': 'blue', 'name': 'fish', 'age': 2}}

Access all attributes of a single node by using square brackets.

In [15]:
H.nodes.attrs[1]

{'color': 'red', 'name': 'horse'}

Access a single attribute of all nodes by specifying a return type.

In [16]:
H.nodes.attrs("color").aslist()

['red', 'blue', 'yellow', 'red', 'blue']

If a node does not have the specified attribute, `None` will be used.

In [17]:
H.nodes.attrs("age").asdict()

{1: None, 2: None, 3: None, 4: 20, 5: 2}

Use the `missing` keyword argument to change the imputed value.

In [18]:
H.nodes.attrs("age", missing=100).asdict()

{1: 100, 2: 100, 3: 100, 4: 20, 5: 2}

## Filtering

`NodeView` objects are aware of existing `NodeStat` objects via the `filterby` method.

In [19]:
H.degree()

{1: 1, 2: 2, 3: 3, 4: 2, 5: 2}

In [20]:
H.nodes.filterby("degree", 2)  # apply the filter to all nodes

NodeView((2, 4, 5))

In [21]:
H.nodes([1, 2, 3]).filterby(
    "degree", 2
)  # apply the filter only to the subset of nodes [1, 2, 3]

NodeView((2,))

Nodes can be filtered by attribute via the `filterby_attr` method.

In [22]:
H.nodes.filterby_attr("color", "red")

NodeView((1, 4))

Since `filterby*` methods return a `NodeView` object, multiple filters can be chained, as well as other `NodeStat` calls. The following call computes the local clustering coefficient of those nodes with degree equal to 2 and "color" attribute equal to "blue", and outputs the result as a dict.

In [23]:
(
    H.nodes.filterby("degree", 2)
    .filterby_attr("color", "blue")
    .clustering_coefficient.asdict()
)

{2: 0.6666666666666666, 5: 1.0}

For example, here is how to access the nodes with maximum degree.

In [24]:
H.nodes.filterby("degree", H.nodes.degree.max())

NodeView((3,))

## Set operations

Another way of chaining multiple results of `filterby*` methods is by using set operations. Indeed, chaining two filters is the same as intersecting the results of two separate calls:

In [25]:
print(H.nodes.filterby("degree", 2).filterby_attr("color", "blue"))
print(H.nodes.filterby("degree", 2) & H.nodes.filterby_attr("color", "blue"))

[2, 5]
[2, 5]


Other set operations are also supported.

In [26]:
nodes1 = H.nodes.filterby("degree", 2)
nodes2 = H.nodes.filterby_attr("color", "blue")
print(f"nodes1 - nodes2 = {nodes1 - nodes2}")
print(f"nodes2 - nodes1 = {nodes2 - nodes1}")
print(f"nodes1 & nodes2 = {nodes1 & nodes2}")
print(f"nodes1 | nodes2 = {nodes1 | nodes2}")
print(f"nodes1 ^ nodes2 = {nodes1 ^ nodes2}")

nodes1 - nodes2 = [4]
nodes2 - nodes1 = []
nodes1 & nodes2 = [2, 5]
nodes1 | nodes2 = [2, 4, 5]
nodes1 ^ nodes2 = [4]


## Edge statistics

Every feature showcased above (lazy evaluation, type conversion, filtering, set operations, and multi objects) is supported for edge-quantity or edge-attribute mappings, via `EdgeStat` objects.

In [34]:
H.edges.order

EdgeStat('order')

In [35]:
H.edges.order.asdict()

{0: 2, 1: 3, 2: 2}

In [36]:
H.edges.filterby("order", 3)

EdgeView((1,))

In [37]:
H.edges.multi(["order", "size"]).aspandas()

,order,size
0,2,3
1,3,4
2,2,3


## User-defined statistics

Suppose during the course of your research you come up with a new node-level statistic. For the purpose of this tutorial, we are going to define a statistic called `user_degree`. The `user_degree` of a node is simply its standard degree times 10.

Since this is also a node-quantity mapping, we would like to give it the same interface as `degree` and all the other `NodeStat`s. The stats package provides a simple way to do this. Simply use the `nodestat_func` decorator.

In [38]:
@xgi.nodestat_func
def user_degree(net, bunch):
    """The user degree of a bunch of nodes in net."""
    return {n: 10 * net.degree(n) for n in bunch}

Now `user_degree` is a valid stat that can be computed on any hypergraph:

In [39]:
H.nodes.user_degree.asdict()

{1: 10, 2: 20, 3: 30, 4: 20, 5: 20}

Every single feature showcased above is available for use with `user_degree`, including filtering nodes and multi stats objects.

In [40]:
H.nodes.filterby("user_degree", 20)

NodeView((2, 4, 5))

The `@xgi.nodestat_func` decorator works on any function or callable that admits two parameters: `net` and `bunch`, where `net` is the network and `bunch` is an iterable of nodes in `net`. Additionally, the function must return a dictionary with pairs of the form `node: value`, where `node` is an element of `bunch`. The library will take care of type conversions, but the output value of this function must always be a dict.

User-defined edge statistics can similarly be defined using the `@xgi.edgestat` decorator.

## Challenge

* Find the number of recipes in `kaggle-whats-cooking` with more than 5 ingredients
* Find the number of ingredients that are only used once. What are they?
* Make a histogram of the edge size. What is the most common number of ingredients in a recipe?
* Extra: Copy the `user_degree` function above and modify it so that it weights each edge by the inverse of its size, i.e.,
$k_i = \sum_{e\in E} {\bf 1}_{i\in e} / |e|$
Output this custom degree for `kaggle-whats-cooking` in dictionary form.
* What is the maximum and minimum number of ingredients?
* What is the most popular ingredient?